# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [22]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [23]:
import requests
headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}
payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [24]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)
response.json()

{'id': 'chatcmpl-CbsFV2r0ls2oXxDTESRxr09B3XjtT',
 'object': 'chat.completion',
 'created': 1763142705,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Fun fact: A day on Venus is longer than a year on Venus. It takes about 243 Earth days to rotate once, but only about 225 days to complete an orbit around the Sun. Want another fun fact?',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 822,
  'total_tokens': 833,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 768,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [25]:
response.json()["choices"][0]["message"]["content"]

'Fun fact: A day on Venus is longer than a year on Venus. It takes about 243 Earth days to rotate once, but only about 225 days to complete an orbit around the Sun. Want another fun fact?'

In [26]:
ollama_api = "http://localhost:11434/api/chat"
ollama_headers = {"Content-Type": "application/json"}
ollama_model = "gpt-oss:120b"
ollama_payload = {
    "model": ollama_model,
    "messages": [
        {
            "role": "user",
            "content": "Tell me a fun fact"
        }
    ],
    "stream": False
}
ollama_payload

{'model': 'gpt-oss:120b',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}],
 'stream': False}

In [27]:
ollama_response = requests.post(
    ollama_api,
    headers=ollama_headers,
    json=ollama_payload
)
ollama_response.json()

{'model': 'gpt-oss:120b',
 'created_at': '2025-11-14T17:52:07.350459424Z',
 'message': {'role': 'assistant',
  'content': '**Fun Fact:**\u202fA single honeybee can visit up to **5,000 flowers** in one day—and in its lifetime it will have made just enough honey to feed **its own weight** in sugar! 🐝🌸\n\nHoneybees are incredibly efficient pollinators. While they’re buzzing from bloom to bloom, they not only gather nectar and pollen for the hive, they also help fertilize the plants they visit, making a huge contribution to the world’s food supply. So next time you see a bee, remember it’s a tiny, winged superhero working around the clock!',
  'thinking': 'User asks "Tell me a fun fact". Simple. Provide a fun fact, maybe about something interesting. Should be concise but engaging. Possibly include explanation. Let\'s give a fun fact.'},
 'done': True,
 'done_reason': 'stop',
 'total_duration': 15393481159,
 'load_duration': 155083140,
 'prompt_eval_count': 72,
 'prompt_eval_duration': 9222

In [28]:
ollama_response.json()["message"]["content"]

'**Fun Fact:**\u202fA single honeybee can visit up to **5,000 flowers** in one day—and in its lifetime it will have made just enough honey to feed **its own weight** in sugar! 🐝🌸\n\nHoneybees are incredibly efficient pollinators. While they’re buzzing from bloom to bloom, they not only gather nectar and pollen for the hive, they also help fertilize the plants they visit, making a huge contribution to the world’s food supply. So next time you see a bee, remember it’s a tiny, winged superhero working around the clock!'

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [29]:
# Create OpenAI client
from openai import OpenAI
openai = OpenAI()
response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])
response.choices[0].message.content

'Fun fact: Honey never spoils. Archaeologists have found pots of honey in ancient Egyptian tombs over 3,000 years old that are still perfectly edible, thanks to honey’s low water content and natural acidity.'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

In [30]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [41]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
response = gemini.chat.completions.create(model="gemini-2.5-pro", messages=[{"role": "user", "content": "Tell me a fun fact"}])
response.choices[0].message.content

"The man who voices Winnie the Pooh (Jim Cummings) also voices Tigger.\n\nThis means that in many scenes where the two characters are talking to each other, he's actually just having a conversation with himself in two completely different voices."

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [32]:
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [33]:
!ollama pull llama3.2

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [34]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [35]:
# Get a fun fact
response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])
response.choices[0].message.content

"Here's one:\n\nDid you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible! Honey's longevity is due to its unique composition, which includes hydrogen peroxide that acts as a natural preservative. This makes it almost impossible for bacteria or other microorganisms to survive, preserving the honey's viability even after thousands of years."

In [36]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud
!ollama pull deepseek-r1:1.5b

pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling aabd4debf0c8: 100% ▕██████████████████▏ 1.1 GB                         
pulling c5ad996bda6e: 100% ▕██████████████████▏  556 B                         
pulling 6e4c38e1172f: 100% ▕██████████████████▏ 1.1 KB                         
pulling f4d24e9138dd: 100% ▕██████████████████▏  148 B                         
pulling a85fe2a2e58e: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [37]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Sure! Here's a fun fact for you: \n\nThe Andromeda Galaxy is the closest galaxy to our own Milky Way, and it's also our neighbor. It can be seen as a faint dot in the background when looking at the sky because we're on the opposite side of it from other stars. Its distance isn't too far compared to the universe! 😊"

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`